## FPGA Memory Map & Address Translation

### Physical Address Space (ARM/Python's view)
The PS (Processing System) sees the following memory layout:

| Region | Address Range | Description |
|--------|--------------|-------------|
| DDR RAM | `0x00000000 – 0x3FFFFFFF` | Main system memory |
| DRAM BRAM (`axi_bram_ctrl_0`) | `0x40000000 – 0x40003FFF` | Data memory (16 KB) |
| IRAM BRAM (`axi_bram_ctrl_1`) | `0x42000000 – 0x42003FFF` | Instruction memory (16 KB) |
| Other IPs | `0x43000000+` | GPIO, etc. |

> The full range `0x40000000 – 0x7FFFFFFF` is reserved for Programmable Logic (PL).

Both BRAMs are **4096 words (16 KB)** each. Physical addresses are resolved automatically by PYNQ via `overlay.mem_dict` — you never need to hardcode them.

---

### RISC-V Address Space (Core's view)
The RISC-V core has its own **byte-addressable, little-endian** address space, mapped by the block design:

| Region | Address Range | Description |
|--------|--------------|-------------|
| Instruction Memory | `0x0000 – 0x0FFF` | Connected to IRAM BRAM |
| Data Memory | `0x1000 – 0x2FFF` | Connected to DRAM BRAM |
| Unmapped | `0x3000+` | Nothing connected |

---

### DRAM Layout & Address Calculations
Within our RISC-V data memory (`0x1000` base):
```
0x1000  ┌─────────────────────────────┐  ← DATA_RAM_START
        │  Reserved (0x40 bytes)      │    Stack, globals, etc. (simulated)
0x1040  ├─────────────────────────────┤  ← DATA_RAM_START + ARRAY_START_OFFSET
        │  Array data (32 × 4 bytes)  │    128 bytes
0x10C0  ├─────────────────────────────┤
        │  ...                        │
0x2000  ├─────────────────────────────┤  ← DATA_RAM_START + STATUS_FLAG_OFFSET
        │  Status flag (DEADBEEF)     │    Written by RISC-V on completion
        └─────────────────────────────┘
```

- **Array base**: Calculated as `array_base = DATA_RAM_START + ARRAY_START_OFFSET` → `0x1000 + 0x40 = 0x1040`
- **Status flag**: Calculated as `status_addr = DATA_RAM_START + STATUS_FLAG_OFFSET` → `0x1000 + 0x1000 = 0x2000`

---

### Pass/Fail Logic
The RISC-V program signals completion by writing the magic number `0xDEADBEEF` to address `0x2000`. Python polls this address and interprets the result:

- **`status == 0xDEADBEEF`** → core finished → read back the array and compare against Python's `sorted()` result
- **Timeout** → core never wrote the flag → something went wrong (bad instructions, reset issue, etc.)
- **Element-wise mismatch** → core ran but sorted incorrectly → likely a pipeline hazard or instruction bug

In [ ]:
from pynq import Overlay, MMIO
import time
import random

# Configuration
ARRAY_SIZE = 32
MAGIC_NUMBER = 0xDEADBEEF

# Memory layout
DATA_RAM_START = 0x1000
ARRAY_START_OFFSET = 0x40
STATUS_FLAG_OFFSET = 0x1000

# Polling
MAX_TIMEOUT = 10  # seconds
POLL_DELAY = 0.001  # 1ms

In [ ]:
# =============================================================
# Load Overlay & Map Hardware
# =============================================================
from pynq import Overlay, MMIO
import time
import random
import numpy as np  

# Configuration
ARRAY_SIZE = 32
MAGIC_NUMBER = 0xDEADBEEF

# Memory layout
DATA_RAM_START = 0x1000
ARRAY_START_OFFSET = 0x40
STATUS_FLAG_OFFSET = 0x1000

# Polling
MAX_TIMEOUT = 10  # seconds
POLL_DELAY = 0.001  # 1ms

BITSTREAM = "design_1_wrapper.bit"

print("Loading Overlay...")
overlay = Overlay(BITSTREAM)  
print("Overlay Loaded.")

print("IPs:", list(overlay.ip_dict.keys()))
print("Mem:", list(overlay.mem_dict.keys()))

# BRAM controllers are in mem_dict
iram_info = overlay.mem_dict['axi_bram_ctrl_1']
dram_info = overlay.mem_dict['axi_bram_ctrl_0']
# CHANGED: bram_iram -> iram, bram_dram -> dram
iram = MMIO(iram_info['phys_addr'], iram_info['addr_range'])
dram = MMIO(dram_info['phys_addr'], dram_info['addr_range'])

# GPIO is in ip_dict
gpio_info = overlay.ip_dict['axi_gpio_0']
gpio = MMIO(gpio_info['phys_addr'], gpio_info['addr_range'])

# Asserts reset by writing 0 to GPIO DATA register, holding core in reset.
def halt_core():
    gpio.write(0x0, 0x0)

# run_core() de-asserts reset (GPIO = 1), allowing core to execute.
def run_core():
    gpio.write(0x0, 0x1)

print("Hardware mapped.")


Loading FPGA Bitstream...
Bitstream loaded successfully!
dict_keys(['axi_bram_ctrl_0', 'axi_bram_ctrl_1', 'PSDDR'])
0x4000


In [ ]:
# =============================================================
# Load Instructions into I-RAM
# =============================================================
print("=" * 70)
print("Loading Instructions...")
print("=" * 70)

halt_core()  # Ensure core is in reset before loading instructions

"""Application Binary Interface names:
a0-a7   = x10-x17  (function arguments/return values)
t0-t6   = x5-x7, x28-x31  (temporary registers)
s0-s11  = x8-x9, x18-x27  (saved registers)
ra      = x1  (return address)
sp      = x2  (stack pointer)
"""
instructions = [
    "00001537", #lui a0, 0x1 -> a0 (x10) = 0x1000 (data RAM base)
    "04050513", #addi a0, a0, 0x40 -> a0 (x10) = 0x1040 (array start).
    "01f00293", #addi t0, zero, 31 -> t0 (x5) = outer loop counter (31 passes)
    "02028e63", #beq t0, zero, done
    "01f00313", #addi t1, zero, 31 -> t1 (x6) = inner loop counter
    "00050593", #addi a1, a0, 0 -> a1 (x11) = current position in array
    #inner_loop:
    "02030463", #beq t1, zero, outer_continue
    "0005a383", #lw t2, 0(a1) -> t2 (x7) = array [i]
    "0045ae03", #lw t3, 4(a1) -> t3 = array [i + 1]
    "007e2eb3", #slt t4, t3, t2 -> t4 = 1 if array [i+1] < array [i], else 0
    "000e8663", #beq t4, zero, no_swap
    "01c5a023", #sw t3, 0(a1) -> array[i] = t3 (was array[i+1])
    "0075a223", #sw t2, 4(a1) -> array [i + 1] = t2 (was array[i])
    #no_swap:
    "00458593", #addi a1, a1, 4 (Increment pointer by 4 bytes)
    "fff30313", #addi t1, t1, -1 (Decrement inner loop counter)
    "fddff06f", #jal zero, inner_loop (Jump back to inner loop)
    #outer_continue (decrement outer loop counter):
    "fff28293", #addi t0, t0, -1
    "fc9ff06f", #jal zero, outer_loop (Jump back to outer loop)
    #done (write status flag with magic number at 0x2000):
    "00002637", #lui a2, 0x2 -> a2 = 0x2000
    "deadf6b7", #lui a3, 0XDEADF -> a3 = 0XDEADF000
    "eef68693", #addi a3, a3, -273 -> a3 = 0XDEADBEEF
    "00d62023", #sw a3, 0(a2) -> Write DEADBEEF to 0x2000
    "ff1ff06f"
]

for i, instr_hex in enumerate(instructions):
    iram.write(i * 4, int(instr_hex, 16))

print(f"Loaded {len(instructions)} instructions to I-ROM")


Loading Instructions...
Loaded 23 instructions to I-ROM


In [ ]:
# =============================================================
# Generate Random Array in DRAM
# =============================================================
print("=" * 70)
print("Generating Test Array...")
print("=" * 70)

# Generate random signed integers
#random.seed(42) #Remove this line for different array each time
test_array = [random.randint(-100, 100) for _ in range(ARRAY_SIZE)]

print(f"Generated {len(test_array)} random integers")
print(f"Range: -100 to 100")
print(f"\nTest array:")
print(test_array)

# Calculate expected result
golden_result = sorted(test_array)
print(f"\nExpected sorted result (first 10):")
print(golden_result[:10])

Generating Test Array...
Generated 32 random integers
Range: -100 to 100

Test array:
[-66, 71, -41, -30, 99, -1, -48, 2, -7, -71, -96, -55, 34, -33, -95, -78, -43, 3, 80, -74, -46, -47, -44, -43, 44, 39, 29, -68, 21, 13, 58, 21]

Expected sorted result (first 10):
[-96, -95, -78, -74, -71, -68, -66, -55, -48, -47]


In [ ]:
# =============================================================
# Inject Test Array into BRAM (D-RAM)
# =============================================================
print("=" * 70)
print("Injecting Test Array into BRAM...")
print("=" * 70)

halt_core()  

# Reset status flag to 0
status_addr = DATA_RAM_START + STATUS_FLAG_OFFSET
dram.write(status_addr, 0x00000000)
print(f"Status flag reset @ {hex(status_addr)}")

# Write array to data RAM
array_base = DATA_RAM_START + ARRAY_START_OFFSET
for i, value in enumerate(test_array):
    # Convert signed to unsigned 32-bit
    unsigned_value = value & 0xFFFFFFFF
    dram.write(array_base + i * 4, unsigned_value) 

print(f"Array injected @ {hex(array_base)}")

# Verify injection (read back first 8 values)
print("\nVerification (first 8 values):")
for i in range(8):
    read_val = dram.read(array_base + i * 4) 
    signed_val = read_val if read_val < 0x80000000 else read_val - 0x100000000
    original = test_array[i]
    match = "OK" if signed_val == original else "FAIL"
    print(f"  [{i}] Expected: {original:4d}, Read: {signed_val:4d} [{match}]")


Injecting Test Array into BRAM...
Status flag reset @ 0x2000
Array injected @ 0x1040

Verification (first 8 values):
  [0] Expected:  -66, Read:  -66 ✓
  [1] Expected:   71, Read:   71 ✓
  [2] Expected:  -41, Read:  -41 ✓
  [3] Expected:  -30, Read:  -30 ✓
  [4] Expected:   99, Read:   99 ✓
  [5] Expected:   -1, Read:   -1 ✓
  [6] Expected:  -48, Read:  -48 ✓
  [7] Expected:    2, Read:    2 ✓


In [ ]:
# =============================================================
# Start RISC-V Core (Release Reset)
# =============================================================
print("=" * 70)
print("Starting RISC-V Core...")
print("=" * 70)

# Assert reset: hold the core in reset before releasing
halt_core() 
print("Reset asserted via halt_core() (GPIO = 0)")
time.sleep(0.02)

# De-assert reset: core starts executing from PC=0
run_core()
print("Reset de-asserted via run_core() (GPIO = 1)")
print("RISC-V core is now running!")
time.sleep(0.02)


Starting RISC-V Core...
Reset asserted (GPIO = 0)
Reset de-asserted (GPIO = 1)
RISC-V core is now running!


In [ ]:
# =============================================================
# GPIO / Reset Signal Monitor
# =============================================================
print("=" * 70)
print("RESET SIGNAL MONITOR")
print("=" * 70)

# Read current GPIO value (register offset 0x0 = DATA register)
current_gpio = gpio.read(0x0)
print(f"Current GPIO DATA register: {current_gpio}")
print(f"  Bit 0 (reset signal): {current_gpio & 0x1}")

# Monitor over time
print("\nMonitoring GPIO over 5 seconds...")
for i in range(10):
    val = gpio.read(0x0)
    rst_val = val & 0x1
    state = 'RUNNING' if rst_val == 1 else 'IN RESET'
    print(f"  t={i*0.5:.1f}s: GPIO={val}, rst_n={rst_val} ({state})")
    time.sleep(0.5)

# Test toggling using halt_core() / run_core()
print("\nTesting reset toggle:")
print("Asserting reset via halt_core()...")
halt_core()  
time.sleep(0.1)
val = gpio.read(0x0)
print(f"  GPIO reads back: {val} (should be 0)")

print("Releasing reset via run_core()...")
run_core()  
time.sleep(0.1)
val = gpio.read(0x0)
print(f"  GPIO reads back: {val} (should be 1)")


RESET SIGNAL MONITOR
Current GPIO DATA register: 1
  Bit 0 (reset signal): 1

Monitoring GPIO over 5 seconds...
  t=0.0s: GPIO=1, rst_n=1 (RUNNING)
  t=0.5s: GPIO=1, rst_n=1 (RUNNING)
  t=1.0s: GPIO=1, rst_n=1 (RUNNING)
  t=1.5s: GPIO=1, rst_n=1 (RUNNING)
  t=2.0s: GPIO=1, rst_n=1 (RUNNING)
  t=2.5s: GPIO=1, rst_n=1 (RUNNING)
  t=3.0s: GPIO=1, rst_n=1 (RUNNING)
  t=3.5s: GPIO=1, rst_n=1 (RUNNING)
  t=4.0s: GPIO=1, rst_n=1 (RUNNING)
  t=4.5s: GPIO=1, rst_n=1 (RUNNING)

Testing reset toggle:
Asserting reset (GPIO=0)...
  GPIO reads back: 0 (should be 0)
Releasing reset (GPIO=1)...
  GPIO reads back: 1 (should be 1)


In [ ]:
# =============================================================
# Poll for Completion
# =============================================================
print("=" * 70)
print("Waiting for Hardware to Complete Sorting...")
print("=" * 70)
print(f"Polling status flag @ {hex(status_addr)}")
print(f"Waiting for magic number: {hex(MAGIC_NUMBER)}")

start_time = time.time()
iterations = 0

while True:
    status = dram.read(status_addr) 

    if status == MAGIC_NUMBER:
        elapsed = time.time() - start_time
        print(f"\nCOMPLETION DETECTED!")
        print(f"  Time: {elapsed:.3f} seconds")
        print(f"  Iterations: {iterations}")
        print(f"  Status flag: {hex(status)}")
        break

    if time.time() - start_time > MAX_TIMEOUT:
        elapsed = time.time() - start_time
        print(f"\nTIMEOUT!")
        print(f"  Time: {elapsed:.3f} seconds")
        print(f"  Final status: {hex(status)}")
        print("  Hardware did not complete in time")
        break

    iterations += 1
    if iterations % 1000 == 0:
        elapsed = time.time() - start_time
        print(f"  Polling... {iterations} iterations ({elapsed:.2f}s) - Status: {hex(status)}")

    time.sleep(POLL_DELAY)


Waiting for Hardware to Complete Sorting...
Polling status flag @ 0x2000
Waiting for magic number: 0xdeadbeef
  Polling... 1000 iterations (1.17s) - Status: 0x0
  Polling... 2000 iterations (2.31s) - Status: 0x0
  Polling... 3000 iterations (3.45s) - Status: 0x0
  Polling... 4000 iterations (4.59s) - Status: 0x0
  Polling... 5000 iterations (5.73s) - Status: 0x0
  Polling... 6000 iterations (6.87s) - Status: 0x0
  Polling... 7000 iterations (8.01s) - Status: 0x0
  Polling... 8000 iterations (9.15s) - Status: 0x0

✗ TIMEOUT!
  Time: 10.000 seconds
  Final status: 0x0
  Hardware did not complete in time


In [ ]:
# =============================================================
# Read Sorted Array from Hardware
# =============================================================
print("=" * 70)
print("Reading Sorted Array from Hardware...")
print("=" * 70)

result_array = []
for i in range(ARRAY_SIZE):
    read_val = dram.read(array_base + i * 4)  # CHANGED: bram_dram -> dram
    # Convert unsigned back to signed
    signed_val = read_val if read_val < 0x80000000 else read_val - 0x100000000
    result_array.append(signed_val)

print(f"Retrieved {len(result_array)} values from hardware")
print(f"\nHardware result (first 10):")
print(result_array[:10])
print(f"\nHardware result (last 10):")
print(result_array[-10:])


Reading Sorted Array from Hardware...
✓ Retrieved 32 values from hardware

Hardware result (first 10):
[66, 0, 91, -62, 47, -57, -24, -93, -60, 50]

Hardware result (last 10):
[46, -30, 42, -18, -4, -62, -42, -2, 37, -74]


In [83]:
#Verification of results
print("=" * 70)
print("VERIFICATION")
print("=" * 70)

print("\nComparison:")
print(f"Original array:  {test_array}")
print(f"Expected sorted: {golden_result}")
print(f"Hardware sorted: {result_array}")

# Check each element
mismatches = []
for i, (expected, actual) in enumerate(zip(golden_result, result_array)):
    if expected != actual:
        mismatches.append((i, expected, actual))

if len(mismatches) == 0:
    print("\n" + "=" * 70)
    print(" TEST PASSED! ")
    print("=" * 70)
    print(f"All {ARRAY_SIZE} values are correctly sorted!")
else:
    print("\n" + "=" * 70)
    print("✗✗✗ TEST FAILED ✗✗✗")
    print("=" * 70)
    print(f"Found {len(mismatches)} mismatches:")
    for idx, expected, actual in mismatches:
        print(f"  Index [{idx:2d}]: Expected {expected:4d}, Got {actual:4d}")

VERIFICATION

Comparison:
Original array:  [66, 2, 91, -62, 47, -57, -24, -93, -60, 50, -98, -59, 66, 17, 98, 40, -21, -31, 47, 32, -68, -73, 46, -30, 42, -18, -4, -62, -42, -2, 37, -74]
Expected sorted: [-98, -93, -74, -73, -68, -62, -62, -60, -59, -57, -42, -31, -30, -24, -21, -18, -4, -2, 2, 17, 32, 37, 40, 42, 46, 47, 47, 50, 66, 66, 91, 98]
Hardware sorted: [66, 0, 91, -62, 47, -57, -24, -93, -60, 50, -98, -59, 66, 17, 98, 40, -21, -256, 47, 32, -68, -73, 46, -30, 42, -18, -4, -62, -42, -2, 37, -74]

✗✗✗ TEST FAILED ✗✗✗
Found 32 mismatches:
  Index [ 0]: Expected  -98, Got   66
  Index [ 1]: Expected  -93, Got    0
  Index [ 2]: Expected  -74, Got   91
  Index [ 3]: Expected  -73, Got  -62
  Index [ 4]: Expected  -68, Got   47
  Index [ 5]: Expected  -62, Got  -57
  Index [ 6]: Expected  -62, Got  -24
  Index [ 7]: Expected  -60, Got  -93
  Index [ 8]: Expected  -59, Got  -60
  Index [ 9]: Expected  -57, Got   50
  Index [10]: Expected  -42, Got  -98
  Index [11]: Expected  -31, G

In [ ]:
# =============================================================
# Debug Helper 1: Memory Dump
# =============================================================
def dump_memory(start_addr, num_words, title="Memory Dump"):
    """Helper function to dump D-RAM memory contents"""
    print(f"\n{title}")
    print("-" * 70)
    for i in range(num_words):
        addr = start_addr + i * 4
        value = dram.read(addr) 
        signed = value if value < 0x80000000 else value - 0x100000000
        print(f"  [{hex(addr)}] word[{i:2d}] = {signed:6d} ({value:#010x})")

print("Debug functions loaded")
print("  Use: dump_memory(address, num_words, title)")
print("  Example: dump_memory(array_base, 32, 'Array Contents')")


✓ Debug functions loaded
  Use: dump_memory(address, num_words, title)
  Example: dump_memory(array_base, 32, 'Array Contents')


In [ ]:
# =============================================================
# Debug Helper 2: Post-Reset GPIO & Memory Check
# =============================================================
print("\n=== POST-RESET DEBUG ===")

print("\n1. Check GPIO:")
gpio_val = gpio.read(0x0)
print(f"   GPIO DATA register: {gpio_val} (should be 1)")

print("\n2. Check first 5 instructions in IRAM:")
for i in range(5):
    val = iram.read(i * 4)  # CHANGED: bram_iram -> iram
    print(f"   IRAM word[{i}] @ offset {i*4:#06x}: {val:#010x}")

print("\n3. Check status flag:")
status = dram.read(status_addr)  # CHANGED: bram_dram -> dram
print(f"   Status @ {hex(status_addr)}: {status:#010x}")

print("\n4. Check array (first 4 values):")
for i in range(4):
    val = dram.read(array_base + i * 4)  # CHANGED: bram_dram -> dram
    signed = val if val < 0x80000000 else val - 0x100000000
    print(f"   Array[{i}] @ {hex(array_base + i*4)}: {signed:4d} ({val:#010x})")

print("\n5. Waiting 2 seconds...")
time.sleep(2)

print("\n6. Check if array changed:")
for i in range(4):
    val = dram.read(array_base + i * 4)  # CHANGED: bram_dram -> dram
    signed = val if val < 0x80000000 else val - 0x100000000
    orig = test_array[i]
    changed = "CHANGED!" if signed != orig else "same"
    print(f"   Array[{i}]: was {orig:4d}, now {signed:4d} - {changed}")

print("\n7. Final status check:")
status = dram.read(status_addr)  # CHANGED: bram_dram -> dram
print(f"   Status: {status:#010x}")


In [ ]:
# =============================================================
# Quick Retest Helper
# =============================================================
def quick_retest(seed=None):
    """Quick function to re-run the bubble sort test with a new random array."""
    print("=" * 70)
    print("QUICK RE-TEST")
    print("=" * 70)

    if seed is not None:
        random.seed(seed)
    new_array = [random.randint(-100, 100) for _ in range(ARRAY_SIZE)]
    print(f"New test array: {new_array}")

    halt_core()

    # Inject new array into D-RAM 
    dram.write(status_addr, 0x00000000)
    for i, value in enumerate(new_array):
        dram.write(array_base + i * 4, value & 0xFFFFFFFF)
    print("Array injected")

    # Reset core 
    halt_core(); time.sleep(0.02)
    run_core();  time.sleep(0.02)
    print("Core restarted via halt_core() + run_core()")

    # Poll for completion
    start = time.time()
    while dram.read(status_addr) != MAGIC_NUMBER:
        if time.time() - start > MAX_TIMEOUT:
            print("Timeout!")
            return
        time.sleep(POLL_DELAY)
    print(f"Complete in {time.time() - start:.3f}s")

    # Read back result
    hw_result = []
    for i in range(ARRAY_SIZE):
        val = dram.read(array_base + i * 4)
        hw_result.append(val if val < 0x80000000 else val - 0x100000000)

    expected = sorted(new_array)
    if hw_result == expected:
        print("TEST PASSED!")
    else:
        print("TEST FAILED!")
        for i, (e, a) in enumerate(zip(expected, hw_result)):
            if e != a:
                print(f"  Mismatch [{i}]: expected {e}, got {a}")

print("Quick retest function loaded")
print("  Use: quick_retest()  or  quick_retest(seed=123)")

#quick_retest()


In [ ]:
# =============================================================
# Verify I-RAM Contents
# =============================================================
print("First 5 instructions in IRAM:")
for i in range(5):
    val = iram.read(i * 4)
    print(f"  [{i}] {val:#010x}")


First 5 instructions in IRAM:
  [0] 0x00001537
  [1] 0x04050513
  [2] 0x01f00293
  [3] 0x02028e63
  [4] 0x01f00313
